In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('filtered_data.csv')
data.set_index('Date', inplace=True)

In [5]:
data

,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,
2000-01-10,C,1460.0,20.500,22.500,267,0.212445,0.508188,0.007420,10949021,100,12,1457.599976,1.001647
2000-01-10,P,1460.0,16.250,17.500,259,0.167732,-0.493139,0.009398,11799565,100,12,1457.599976,1.001647
2000-01-11,C,1440.0,20.000,21.250,14,0.212868,0.513452,0.007866,11623411,100,11,1438.560059,1.001001
2000-01-11,P,1440.0,16.375,17.625,137,0.183284,-0.486487,0.009136,10563021,100,11,1438.560059,1.001001
2000-01-12,C,1430.0,18.250,19.500,0,0.185919,0.546470,0.009476,11778044,100,10,1432.250000,0.998429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,P,4510.0,12.300,12.400,5085,0.121828,-0.522582,0.013857,155631078,100,1,4507.660156,1.000519
2023-08-31,C,4510.0,32.100,32.400,39,0.096006,0.527107,0.005298,156108279,100,11,4507.660156,1.000519
2023-08-31,P,4510.0,26.700,27.000,40,0.093054,-0.472241,0.005465,156108338,100,11,4507.660156,1.000519
